In [13]:
import unittest
from Stress_Test_HW2_20201117 import SimpleRegressionRiskGeneratorSQL
from Stress_Test_HW2_20201117 import LassoRegressionRiskGeneratorSQL
import os
import pandas as pd
import numpy as np

class Test_SQL_functionality(unittest.TestCase):
    
    def test_FundsSQLupdate_SimpleRegression(self):
        os.environ['USER'] = 'parag'
        os.environ['PASSWORD'] = 'mypass'
        username = os.getenv('USER')
        password = os.getenv('PASSWORD')
        factors = ['morningstar msci emerging markets','s&p 500 pr', 'bbgbarc us treasury tr usd(1972)',
           'libor 3 mon interbank eurodollar inv tr','bbgbarc us credit tr usd']
        # first, we test class the SimpleRegressionRiskGeneratorSQL
        x = SimpleRegressionRiskGeneratorSQL(risk_generator_name = 'Regression', risk_factors = factors, username = username, password = password)
        x.set_up(hyperparameters = {}, data_path = {'index': {'morningstar msci emerging markets': 'Emerging Markets.csv',
                                                              's&p 500 pr': 'S&P 500.csv',
                                                              'bbgbarc us treasury tr usd(1972)': 'BBgBarc US Treasury.csv',
                                                              'libor 3 mon interbank eurodollar inv tr': 'LIBOR 3 Mon Interbank Eurodollar Inv.csv',
                                                              'bbgbarc us credit tr usd': 'BBgBarc US Credit.csv'}
                                                    ,'fund': 'Fund.csv'})
        x.update_raw_data()
        funds_data = pd.read_csv(x._data_path.get('fund'),sep=',', thousands=',')
        funds_data_SQL = x._conn.get_dataframe(table_name = 'funds', schema = 'Funds') 
        self.assertTrue(funds_data.equals(funds_data_SQL))
    
    def test_FactorsSQLupdate_SimpleRegression(self):
        os.environ['USER'] = 'parag'
        os.environ['PASSWORD'] = 'mypass'
        username = os.getenv('USER')
        password = os.getenv('PASSWORD')
        factors = ['morningstar msci emerging markets','s&p 500 pr', 'bbgbarc us treasury tr usd(1972)',
           'libor 3 mon interbank eurodollar inv tr','bbgbarc us credit tr usd']
        # first, we test class the SimpleRegressionRiskGeneratorSQL
        x = SimpleRegressionRiskGeneratorSQL(risk_generator_name = 'Regression', risk_factors = factors, username = username, password = password)
        x.set_up(hyperparameters = {}, data_path = {'index': {'morningstar msci emerging markets': 'Emerging Markets.csv',
                                                              's&p 500 pr': 'S&P 500.csv',
                                                              'bbgbarc us treasury tr usd(1972)': 'BBgBarc US Treasury.csv',
                                                              'libor 3 mon interbank eurodollar inv tr': 'LIBOR 3 Mon Interbank Eurodollar Inv.csv',
                                                              'bbgbarc us credit tr usd': 'BBgBarc US Credit.csv'}
                                                    ,'fund': 'Fund.csv'})
        x.update_raw_data()
        for factor, path in x._data_path.get('index').items(): # its a dict {factor name: path of csv file corresponding} factor name more covenient than index name
            factor_data = pd.read_csv(path,sep=',', thousands=',')
            factor_data.rename(columns={'Unnamed: 0': 'Date', 
                                        factor_data.columns[1]: factor_data.columns[1].split(' -')[0]}, inplace=True) 
            self.assertTrue(factor_data.equals(x._conn.get_dataframe(table_name = factor, schema = 'factors'))) # indiv_factors_data is a dict
            
    def test_results_SQL_SimpleRegression(self):
        os.environ['USER'] = 'parag'
        os.environ['PASSWORD'] = 'mypass'
        username = os.getenv('USER')
        password = os.getenv('PASSWORD')
        factors = ['morningstar msci emerging markets','s&p 500 pr', 'bbgbarc us treasury tr usd(1972)',
           'libor 3 mon interbank eurodollar inv tr','bbgbarc us credit tr usd']
        # first, we test class the SimpleRegressionRiskGeneratorSQL
        x = SimpleRegressionRiskGeneratorSQL(risk_generator_name = 'Regression', risk_factors = factors, username = username, password = password)
        x.set_up(hyperparameters = {}, data_path = {'index': {'morningstar msci emerging markets': 'Emerging Markets.csv',
                                                              's&p 500 pr': 'S&P 500.csv',
                                                              'bbgbarc us treasury tr usd(1972)': 'BBgBarc US Treasury.csv',
                                                              'libor 3 mon interbank eurodollar inv tr': 'LIBOR 3 Mon Interbank Eurodollar Inv.csv',
                                                              'bbgbarc us credit tr usd': 'BBgBarc US Credit.csv'}
                                                    ,'fund': 'Fund.csv'})
        x.update_raw_data()
        x.split_train_test_data()
        x.fit()
        x.predict()
        x.betas_fund()
        x.publish_result()
        betas_from_SQL = x._conn.get_dataframe(table_name = 'results', schema = 'Results')
        betas_from_csv = pd.DataFrame.from_dict(x._betas_fund)
        betas_from_csv.insert(0, 'Factor', x._risk_factors)
        self.assertTrue(betas_from_SQL.equals(betas_from_csv))
        
    def test_FundsSQLupdate_LassoRegression(self):
            os.environ['USER'] = 'parag'
            os.environ['PASSWORD'] = 'mypass'
            username = os.getenv('USER')
            password = os.getenv('PASSWORD')
            factors = ['morningstar msci emerging markets','s&p 500 pr', 'bbgbarc us treasury tr usd(1972)',
               'libor 3 mon interbank eurodollar inv tr','bbgbarc us credit tr usd']
            # first, we test class the SimpleRegressionRiskGeneratorSQL
            x = LassoRegressionRiskGeneratorSQL(risk_generator_name = 'Lasso', risk_factors = factors, username = username, password = password)
            x.set_up(hyperparameters = {'alpha': 0.6, 'max_iter': 100000}, data_path = {'index': {'morningstar msci emerging markets': 'Emerging Markets.csv',
                                                                  's&p 500 pr': 'S&P 500.csv',
                                                                  'bbgbarc us treasury tr usd(1972)': 'BBgBarc US Treasury.csv',
                                                                  'libor 3 mon interbank eurodollar inv tr': 'LIBOR 3 Mon Interbank Eurodollar Inv.csv',
                                                                  'bbgbarc us credit tr usd': 'BBgBarc US Credit.csv'}
                                                        ,'fund': 'Fund.csv'})
            x.update_raw_data()
            funds_data = pd.read_csv(x._data_path.get('fund'),sep=',', thousands=',')
            funds_data_SQL = x._conn.get_dataframe(table_name = 'funds', schema = 'Funds') 
            self.assertTrue(funds_data.equals(funds_data_SQL))
    
    def test_FactorsSQLupdate_LassoRegression(self):
            os.environ['USER'] = 'parag'
            os.environ['PASSWORD'] = 'mypass'
            username = os.getenv('USER')
            password = os.getenv('PASSWORD')
            factors = ['morningstar msci emerging markets','s&p 500 pr', 'bbgbarc us treasury tr usd(1972)',
               'libor 3 mon interbank eurodollar inv tr','bbgbarc us credit tr usd']
            # first, we test class the SimpleRegressionRiskGeneratorSQL
            x = LassoRegressionRiskGeneratorSQL(risk_generator_name = 'Lasso', risk_factors = factors, username = username, password = password)
            x.set_up(hyperparameters = {'alpha': 0.6, 'max_iter': 100000}, data_path = {'index': {'morningstar msci emerging markets': 'Emerging Markets.csv',
                                                                  's&p 500 pr': 'S&P 500.csv',
                                                                  'bbgbarc us treasury tr usd(1972)': 'BBgBarc US Treasury.csv',
                                                                  'libor 3 mon interbank eurodollar inv tr': 'LIBOR 3 Mon Interbank Eurodollar Inv.csv',
                                                                  'bbgbarc us credit tr usd': 'BBgBarc US Credit.csv'}
                                                        ,'fund': 'Fund.csv'})
            x.update_raw_data()
            for factor, path in x._data_path.get('index').items(): # its a dict {factor name: path of csv file corresponding} factor name more covenient than index name
                factor_data = pd.read_csv(path,sep=',', thousands=',')
                factor_data.rename(columns={'Unnamed: 0': 'Date', 
                                            factor_data.columns[1]: factor_data.columns[1].split(' -')[0]}, inplace=True) 
                self.assertTrue(factor_data.equals(x._conn.get_dataframe(table_name = factor, schema = 'factors'))) # indiv_factors_data is a dict

    def test_ResultsSQLupdate_LassoRegression(self):
            os.environ['USER'] = 'parag'
            os.environ['PASSWORD'] = 'mypass'
            username = os.getenv('USER')
            password = os.getenv('PASSWORD')
            factors = ['morningstar msci emerging markets','s&p 500 pr', 'bbgbarc us treasury tr usd(1972)',
               'libor 3 mon interbank eurodollar inv tr','bbgbarc us credit tr usd']
            # first, we test class the SimpleRegressionRiskGeneratorSQL
            x = LassoRegressionRiskGeneratorSQL(risk_generator_name = 'Lasso', risk_factors = factors, username = username, password = password)
            x.set_up(hyperparameters = {'alpha': 0.6, 'max_iter': 100000}, data_path = {'index': {'morningstar msci emerging markets': 'Emerging Markets.csv',
                                                                  's&p 500 pr': 'S&P 500.csv',
                                                                  'bbgbarc us treasury tr usd(1972)': 'BBgBarc US Treasury.csv',
                                                                  'libor 3 mon interbank eurodollar inv tr': 'LIBOR 3 Mon Interbank Eurodollar Inv.csv',
                                                                  'bbgbarc us credit tr usd': 'BBgBarc US Credit.csv'}
                                                        ,'fund': 'Fund.csv'})
            x.update_raw_data()
            x.split_train_test_data()
            x.fit()
            x.predict()
            x.betas_fund()
            x.publish_result()
            betas_from_SQL = x._conn.get_dataframe(table_name = 'results', schema = 'Results')
            betas_from_csv = pd.DataFrame.from_dict(x._betas_fund)
            betas_from_csv.insert(0, 'Factor', x._risk_factors)
            self.assertTrue(betas_from_SQL.equals(betas_from_csv))

    def test_FundsSQLupdate_RidgeRegression(self):
            os.environ['USER'] = 'parag'
            os.environ['PASSWORD'] = 'mypass'
            username = os.getenv('USER')
            password = os.getenv('PASSWORD')
            factors = ['morningstar msci emerging markets','s&p 500 pr', 'bbgbarc us treasury tr usd(1972)',
               'libor 3 mon interbank eurodollar inv tr','bbgbarc us credit tr usd']
            # first, we test class the SimpleRegressionRiskGeneratorSQL
            x = LassoRegressionRiskGeneratorSQL(risk_generator_name = 'Lasso', risk_factors = factors, username = username, password = password)
            x.set_up(hyperparameters = {'alpha': 0.6, 'max_iter': 100000}, data_path = {'index': {'morningstar msci emerging markets': 'Emerging Markets.csv',
                                                                  's&p 500 pr': 'S&P 500.csv',
                                                                  'bbgbarc us treasury tr usd(1972)': 'BBgBarc US Treasury.csv',
                                                                  'libor 3 mon interbank eurodollar inv tr': 'LIBOR 3 Mon Interbank Eurodollar Inv.csv',
                                                                  'bbgbarc us credit tr usd': 'BBgBarc US Credit.csv'}
                                                        ,'fund': 'Fund.csv'})
            x.update_raw_data()
            funds_data = pd.read_csv(x._data_path.get('fund'),sep=',', thousands=',')
            funds_data_SQL = x._conn.get_dataframe(table_name = 'funds', schema = 'Funds') 
            self.assertTrue(funds_data.equals(funds_data_SQL))
    
    def test_FactorsSQLupdate_LassoRegression(self):
            os.environ['USER'] = 'parag'
            os.environ['PASSWORD'] = 'mypass'
            username = os.getenv('USER')
            password = os.getenv('PASSWORD')
            factors = ['morningstar msci emerging markets','s&p 500 pr', 'bbgbarc us treasury tr usd(1972)',
               'libor 3 mon interbank eurodollar inv tr','bbgbarc us credit tr usd']
            # first, we test class the SimpleRegressionRiskGeneratorSQL
            x = LassoRegressionRiskGeneratorSQL(risk_generator_name = 'Lasso', risk_factors = factors, username = username, password = password)
            x.set_up(hyperparameters = {'alpha': 0.6, 'max_iter': 100000}, data_path = {'index': {'morningstar msci emerging markets': 'Emerging Markets.csv',
                                                                  's&p 500 pr': 'S&P 500.csv',
                                                                  'bbgbarc us treasury tr usd(1972)': 'BBgBarc US Treasury.csv',
                                                                  'libor 3 mon interbank eurodollar inv tr': 'LIBOR 3 Mon Interbank Eurodollar Inv.csv',
                                                                  'bbgbarc us credit tr usd': 'BBgBarc US Credit.csv'}
                                                        ,'fund': 'Fund.csv'})
            x.update_raw_data()
            for factor, path in x._data_path.get('index').items(): # its a dict {factor name: path of csv file corresponding} factor name more covenient than index name
                factor_data = pd.read_csv(path,sep=',', thousands=',')
                factor_data.rename(columns={'Unnamed: 0': 'Date', 
                                            factor_data.columns[1]: factor_data.columns[1].split(' -')[0]}, inplace=True) 
                self.assertTrue(factor_data.equals(x._conn.get_dataframe(table_name = factor, schema = 'factors'))) # indiv_factors_data is a dict

    def test_ResultsSQLupdate_LassoRegression(self):
            os.environ['USER'] = 'parag'
            os.environ['PASSWORD'] = 'mypass'
            username = os.getenv('USER')
            password = os.getenv('PASSWORD')
            factors = ['morningstar msci emerging markets','s&p 500 pr', 'bbgbarc us treasury tr usd(1972)',
               'libor 3 mon interbank eurodollar inv tr','bbgbarc us credit tr usd']
            # first, we test class the SimpleRegressionRiskGeneratorSQL
            x = LassoRegressionRiskGeneratorSQL(risk_generator_name = 'Lasso', risk_factors = factors, username = username, password = password)
            x.set_up(hyperparameters = {'alpha': 0.6, 'max_iter': 100000}, data_path = {'index': {'morningstar msci emerging markets': 'Emerging Markets.csv',
                                                                  's&p 500 pr': 'S&P 500.csv',
                                                                  'bbgbarc us treasury tr usd(1972)': 'BBgBarc US Treasury.csv',
                                                                  'libor 3 mon interbank eurodollar inv tr': 'LIBOR 3 Mon Interbank Eurodollar Inv.csv',
                                                                  'bbgbarc us credit tr usd': 'BBgBarc US Credit.csv'}
                                                        ,'fund': 'Fund.csv'})
            x.update_raw_data()
            x.split_train_test_data()
            x.fit()
            x.predict()
            x.betas_fund()
            x.publish_result()
            betas_from_SQL = x._conn.get_dataframe(table_name = 'results', schema = 'Results')
            betas_from_csv = pd.DataFrame.from_dict(x._betas_fund)
            betas_from_csv.insert(0, 'Factor', x._risk_factors)
            self.assertTrue(betas_from_SQL.equals(betas_from_csv))

unittest.main(argv=[''], verbosity=2, exit=False)        
        
    
        
    
        
        

test_FactorsSQLupdate_LassoRegression (__main__.Test_SQL_functionality) ... 

Enter schema(optional): 

morningstar msci emerging markets exists in schema Factors
s&p 500 pr exists in schema Factors
bbgbarc us treasury tr usd(1972) exists in schema Factors
libor 3 mon interbank eurodollar inv tr exists in schema Factors
bbgbarc us credit tr usd exists in schema Factors
funds exists in schema Funds
results exists in schema Results


ok
test_FactorsSQLupdate_SimpleRegression (__main__.Test_SQL_functionality) ... 

Enter schema(optional): 

morningstar msci emerging markets exists in schema Factors
s&p 500 pr exists in schema Factors
bbgbarc us treasury tr usd(1972) exists in schema Factors
libor 3 mon interbank eurodollar inv tr exists in schema Factors
bbgbarc us credit tr usd exists in schema Factors
funds exists in schema Funds
results exists in schema Results


ok
test_FundsSQLupdate_LassoRegression (__main__.Test_SQL_functionality) ... 

...

ok
test_ResultsSQLupdate_LassoRegression (__main__.Test_SQL_functionality) ... 

Enter schema(optional): 

morningstar msci emerging markets exists in schema Factors
s&p 500 pr exists in schema Factors
bbgbarc us treasury tr usd(1972) exists in schema Factors
libor 3 mon interbank eurodollar inv tr exists in schema Factors
bbgbarc us credit tr usd exists in schema Factors
funds exists in schema Funds
results exists in schema Results


ok
test_results_SQL_SimpleRegression (__main__.Test_SQL_functionality) ... 

Enter schema(optional): 

morningstar msci emerging markets exists in schema Factors
s&p 500 pr exists in schema Factors
bbgbarc us treasury tr usd(1972) exists in schema Factors
libor 3 mon interbank eurodollar inv tr exists in schema Factors
bbgbarc us credit tr usd exists in schema Factors
funds exists in schema Funds
results exists in schema Results


ok

----------------------------------------------------------------------
Ran 6 tests in 21.915s

OK
